In [1]:
import nibabel as nib
import numpy as np
import pandas as pd
import os
from pathlib import Path

In [2]:
# Base directory
base_dir = '/BICNAS2/group-northoff/jkokino/normalized'
output_dir = '/home/jkokino/meditation_project/timeseries_extracted_self'
master_atlas = '/home/jkokino/meditation_project/templates/glasser_files/self_atlas_4mm.nii.gz'
os.makedirs(output_dir, exist_ok=True)

# Task types
tasks = ['ca1', 'ca2', 'conc1', 'conc2', 'metta1', 'metta2']

In [3]:
def extract_roi_timeseries(bold_file, roi_atlas_file):
    """
    Extract timeseries for each ROI from the atlas
    """
    # Load functional data
    bold_img = nib.load(bold_file)
    bold_data = bold_img.get_fdata()  # (x, y, z, time)
    
    print(f"    BOLD shape: {bold_data.shape}")
    
    # Load ROI atlas
    roi_img = nib.load(roi_atlas_file)
    roi_atlas = roi_img.get_fdata()  # (x, y, z) with ROI numbers
    
    print(f"    ROI atlas shape: {roi_atlas.shape}")
    
    # Get unique ROI numbers (excluding 0 which is background)
    roi_numbers = np.unique(roi_atlas)
    roi_numbers = roi_numbers[roi_numbers > 0]  # Remove 0 (background)
    
    print(f"    Found {len(roi_numbers)} ROIs")
    
    # Extract timeseries for each ROI
    roi_timeseries = {}
    
    for roi_num in roi_numbers:
        # Create mask for this ROI
        roi_mask = (roi_atlas == roi_num)
        n_voxels = np.sum(roi_mask)
        
        if n_voxels == 0:
            print(f"      Warning: ROI {int(roi_num)} has 0 voxels!")
            continue
        
        # Average voxels in this ROI across time
        # bold_data[roi_mask, :] gets all timepoints for voxels in this ROI
        timeseries = bold_data[roi_mask, :].mean(axis=0)
        roi_timeseries[int(roi_num)] = timeseries
        
        if int(roi_num) <= 3:  # Print first few for debugging
            print(f"      ROI {int(roi_num)}: {n_voxels} voxels, {len(timeseries)} timepoints")
    
    return roi_timeseries

print("Function defined!")

Function defined!


In [5]:
# Process all groups and subjects
for group in ['controls', 'meditators']:
    
    group_dir = os.path.join(base_dir, group)
    
    # Get all subject directories
    subject_dirs = sorted([d for d in os.listdir(group_dir) 
                          if d.startswith('sub-') and 
                          os.path.isdir(os.path.join(group_dir, d))])
    
    print(f"Found {len(subject_dirs)} subjects")
    
    for subject in subject_dirs:
        print(f"\n  Processing: {subject}")

        func_dir = os.path.join(group_dir, subject, 'func')
        
        # Create output directory for this subject
        subject_output = os.path.join(output_dir, group, subject)
        os.makedirs(subject_output, exist_ok=True)
        
        # Process each task
        for task in tasks:
            bold_file = os.path.join(func_dir, f'{subject}_task-{task}_bold_space-MNI.nii.gz')            


            try:
                # Extract ROI timeseries 
                roi_timeseries = extract_roi_timeseries(bold_file, master_atlas)
                
                # Save as single CSV
                df = pd.DataFrame(roi_timeseries)
                df.index.name = 'Timepoint'
                csv_file = os.path.join(subject_output, f'{subject}_{task}_timeseries.csv')
                df.to_csv(csv_file)

                
                print(f"     Saved {len(roi_timeseries)} ROI timeseries for {task}")
                
            except Exception as e:
                print(f"     Error in {task}: {str(e)}")
                import traceback
                traceback.print_exc()
        
print(f"Output saved to: {output_dir}")

Found 20 subjects

  Processing: sub-048


    BOLD shape: (64, 76, 64, 194)
    ROI atlas shape: (64, 76, 64)
    Found 327 ROIs
      ROI 1: 6 voxels, 194 timepoints
      ROI 2: 10 voxels, 194 timepoints
      ROI 3: 10 voxels, 194 timepoints
     Saved 327 ROI timeseries for ca1
    BOLD shape: (64, 76, 64, 194)
    ROI atlas shape: (64, 76, 64)
    Found 327 ROIs
      ROI 1: 6 voxels, 194 timepoints
      ROI 2: 10 voxels, 194 timepoints
      ROI 3: 10 voxels, 194 timepoints
     Saved 327 ROI timeseries for ca2
     Error in conc1: Compressed file ended before the end-of-stream marker was reached


Traceback (most recent call last):
  File "/tmp/ipykernel_18279/575399939.py", line 29, in <module>
    roi_timeseries = extract_roi_timeseries(bold_file, master_atlas)
  File "/tmp/ipykernel_18279/3691283324.py", line 7, in extract_roi_timeseries
    bold_data = bold_img.get_fdata()  # (x, y, z, time)
  File "/home/jkokino/miniconda3/envs/fmri/lib/python3.8/site-packages/nibabel/dataobj_images.py", line 373, in get_fdata
    data = np.asanyarray(self._dataobj, dtype=dtype)
  File "/home/jkokino/miniconda3/envs/fmri/lib/python3.8/site-packages/nibabel/arrayproxy.py", line 457, in __array__
    arr = self._get_scaled(dtype=dtype, slicer=())
  File "/home/jkokino/miniconda3/envs/fmri/lib/python3.8/site-packages/nibabel/arrayproxy.py", line 424, in _get_scaled
    scaled = apply_read_scaling(self._get_unscaled(slicer=slicer), scl_slope, scl_inter)
  File "/home/jkokino/miniconda3/envs/fmri/lib/python3.8/site-packages/nibabel/arrayproxy.py", line 394, in _get_unscaled
    return array_from

    BOLD shape: (64, 76, 64, 194)
    ROI atlas shape: (64, 76, 64)
    Found 327 ROIs
      ROI 1: 6 voxels, 194 timepoints
      ROI 2: 10 voxels, 194 timepoints
      ROI 3: 10 voxels, 194 timepoints
     Saved 327 ROI timeseries for conc2
    BOLD shape: (64, 76, 64, 194)
    ROI atlas shape: (64, 76, 64)
    Found 327 ROIs
      ROI 1: 6 voxels, 194 timepoints
      ROI 2: 10 voxels, 194 timepoints
      ROI 3: 10 voxels, 194 timepoints
     Saved 327 ROI timeseries for metta1
    BOLD shape: (64, 76, 64, 194)
    ROI atlas shape: (64, 76, 64)
    Found 327 ROIs
      ROI 1: 6 voxels, 194 timepoints
      ROI 2: 10 voxels, 194 timepoints
      ROI 3: 10 voxels, 194 timepoints
     Saved 327 ROI timeseries for metta2

  Processing: sub-050
    BOLD shape: (64, 76, 64, 194)
    ROI atlas shape: (64, 76, 64)
    Found 327 ROIs
      ROI 1: 6 voxels, 194 timepoints
      ROI 2: 10 voxels, 194 timepoints
      ROI 3: 10 voxels, 194 timepoints
     Saved 327 ROI timeseries for ca1
    B

Traceback (most recent call last):
  File "/home/jkokino/miniconda3/envs/fmri/lib/python3.8/site-packages/nibabel/loadsave.py", line 100, in load
    stat_result = os.stat(filename)
FileNotFoundError: [Errno 2] No such file or directory: '/BICNAS2/group-northoff/jkokino/normalized/meditators/sub-043/func/sub-043_task-ca1_bold_space-MNI.nii.gz'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/ipykernel_18279/575399939.py", line 29, in <module>
    roi_timeseries = extract_roi_timeseries(bold_file, master_atlas)
  File "/tmp/ipykernel_18279/3691283324.py", line 6, in extract_roi_timeseries
    bold_img = nib.load(bold_file)
  File "/home/jkokino/miniconda3/envs/fmri/lib/python3.8/site-packages/nibabel/loadsave.py", line 102, in load
    raise FileNotFoundError(f"No such file or no access: '{filename}'")
FileNotFoundError: No such file or no access: '/BICNAS2/group-northoff/jkokino/normalized/meditators/sub-043/func/sub-

    BOLD shape: (64, 76, 64, 194)
    ROI atlas shape: (64, 76, 64)
    Found 327 ROIs
      ROI 1: 6 voxels, 194 timepoints
      ROI 2: 10 voxels, 194 timepoints
      ROI 3: 10 voxels, 194 timepoints
     Saved 327 ROI timeseries for ca1
    BOLD shape: (64, 76, 64, 194)
    ROI atlas shape: (64, 76, 64)
    Found 327 ROIs
      ROI 1: 6 voxels, 194 timepoints
      ROI 2: 10 voxels, 194 timepoints
      ROI 3: 10 voxels, 194 timepoints
     Saved 327 ROI timeseries for ca2
    BOLD shape: (64, 76, 64, 194)
    ROI atlas shape: (64, 76, 64)
    Found 327 ROIs
      ROI 1: 6 voxels, 194 timepoints
      ROI 2: 10 voxels, 194 timepoints
      ROI 3: 10 voxels, 194 timepoints
     Saved 327 ROI timeseries for conc1
    BOLD shape: (64, 76, 64, 194)
    ROI atlas shape: (64, 76, 64)
    Found 327 ROIs
      ROI 1: 6 voxels, 194 timepoints
      ROI 2: 10 voxels, 194 timepoints
      ROI 3: 10 voxels, 194 timepoints
     Saved 327 ROI timeseries for conc2
    BOLD shape: (64, 76, 64, 194

In [13]:
# Test with one subject
test_file = '/home/jkokino/meditation_project/data/preprocessed/BIDS_nifti/controls/sub-048/func/sub-048_task-ca1_bold.nii.gz'

if os.path.exists(test_file):
    print("Testing with sub-048...")
    signal = extract_global_signal(test_file)
    print(f"\nGlobal signal shape: {signal.shape}")
    print(f"First 10 timepoints: {signal[:10]}")
else:
    print(f"Test file not found at: {test_file}")

Testing with sub-048...
  Data shape: (64, 64, 25, 194)
  Brain voxels: 8 (0.0%)

Global signal shape: (194,)
First 10 timepoints: [1139.5   1108.875 1055.5   1079.125 1043.875  961.125 1050.75  1036.875
 1057.625  969.5  ]
